# MY NLP MODELING EXERCISES

In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import nltk
import unicodedata
import re
import env
import acquire
import prepare
from wordcloud import WordCloud
import nltk.sentiment
pd.set_option('display.max_colwidth', None)
from time import strftime
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:
#acquire data from spam_db

from env import user, password, host
raw = pd.read_csv("spam_clean - spam_clean.csv", index_col='label')

raw.head()

,text
label,
ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat..."
ham,Ok lar... Joking wif u oni...
spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive entry question(std txt rate)T&C's apply 08452810075over18's
ham,U dun say so early hor... U c already then say...
ham,"Nah I don't think he goes to usf, he lives around here though"


In [3]:
raw['cleaned']=raw.text.apply(prepare.basic_clean).apply(prepare.tokenize)\
.apply(prepare.remove_stopwords, extra_words=[], 
                                   exclude_words=[])
raw['stemmed'] = raw.cleaned.apply(prepare.stem)
raw['lemmatized']= raw.cleaned.apply(prepare.lemmatize)
raw = raw.drop(columns=['text','cleaned','stemmed'])
raw.head()

,lemmatized
label,
ham,go jurong point crazy available bugis n great world la e buffet cine got amore wat
ham,ok lar joking wif u oni
spam,free entry 2 wkly comp win fa cup final tkts 21st may 2005 text fa 87121 receive entry questionstd txt ratetcs apply 08452810075over18s
ham,u dun say early hor u c already say
ham,nah dont think go usf life around though


In [4]:
raw.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5572 entries, ham to ham
Data columns (total 1 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   lemmatized  5572 non-null   object
dtypes: object(1)
memory usage: 87.1+ KB


In [6]:
raw.shape

(5572, 1)

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score


df = raw
df.head()

tfidf = TfidfVectorizer()
X = tfidf.fit_transform(df.lemmatized)
y = df.index

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=.2)

train = pd.DataFrame(dict(actual=y_train))
test = pd.DataFrame(dict(actual=y_test))

lm = LogisticRegression().fit(X_train, y_train)

train['predicted'] = lm.predict(X_train)
test['predicted'] = lm.predict(X_test)

In [9]:
print('Accuracy: {:.2%}'.format(accuracy_score(train.actual, train.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(train.predicted, train.actual))
print('---')
print(classification_report(train.actual, train.predicted))

Accuracy: 96.39%
---
Confusion Matrix
actual      ham  spam
predicted            
ham        3853   155
spam          6   443
---
              precision    recall  f1-score   support

         ham       0.96      1.00      0.98      3859
        spam       0.99      0.74      0.85       598

    accuracy                           0.96      4457
   macro avg       0.97      0.87      0.91      4457
weighted avg       0.96      0.96      0.96      4457



# I'm really interested in applying SMOTE to the spam data to equal out the number of observations for either category

In [10]:
print('Accuracy: {:.2%}'.format(accuracy_score(test.actual, test.predicted)))
print('---')
print('Confusion Matrix')
print(pd.crosstab(test.predicted, test.actual))
print('---')
print(classification_report(test.actual, test.predicted))


Accuracy: 95.07%
---
Confusion Matrix
actual     ham  spam
predicted           
ham        964    53
spam         2    96
---
              precision    recall  f1-score   support

         ham       0.95      1.00      0.97       966
        spam       0.98      0.64      0.78       149

    accuracy                           0.95      1115
   macro avg       0.96      0.82      0.87      1115
weighted avg       0.95      0.95      0.95      1115

